# ETL pipeline

In [1]:
from datetime import datetime, timedelta
from ETL_pipeline_functions import week_ago, iso_8601_to_unix, token_exchange, request_activities, request_weather, request_splits, request_zones, append_requests

In [2]:
now = datetime.now()
test_date = now - timedelta(weeks = 2)
test_date = iso_8601_to_unix(test_date)

In [3]:
def ETL_pipeline(test_date):
    # start_date = iso_8601_to_unix(week_ago())

    activities = request_activities(test_date)
    append_requests(activities, 'activities.csv')

    activity_ids = list(map(lambda activity: activity['activity_id'], activities))

    weather_list = request_weather(activity_ids)
    append_requests(weather_list, 'activity_weather.csv')

    activity_splits = request_splits(activity_ids)
    append_requests(activity_splits, 'activity_splits.csv')

    activity_zones = request_zones(activity_ids)
    append_requests(activity_zones, 'activity_zones.csv')

    return print("ETL pipeline complete")

In [4]:
ETL_pipeline(test_date)

activities.csv appended
activity_weather.csv appended
activity_splits.csv appended
activity_zones.csv appended
ETL pipeline complete
